In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,31479
2,Huelva,Confirmados PDIA 14 días,1464
3,Huelva,Tasa PDIA 14 días,"285,28557787867567"
4,Huelva,Confirmados PDIA 7 días,751
5,Huelva,Total Confirmados,31659
6,Huelva,Curados,28956
7,Huelva,Fallecidos,383


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  31479.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  8381.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 516 personas en los últimos 7 días 

Un positivo PCR cada 276 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,31479.0,751.0,1464.0,513170.0,146.345266,285.285578,148.0
Huelva-Costa,18115.0,387.0,753.0,289548.0,133.656596,260.060508,87.0
Huelva (capital),8381.0,279.0,521.0,143837.0,193.969563,362.215563,61.0
Condado-Campiña,10227.0,274.0,570.0,156231.0,175.381326,364.844365,39.0
Sierra de Huelva-Andévalo Central,2788.0,86.0,134.0,67391.0,127.613480,198.839608,21.0
Almonte,1836.0,76.0,140.0,24507.0,310.115477,571.265353,19.0
Aljaraque,1107.0,45.0,74.0,21474.0,209.555742,344.602775,14.0
Valverde del Camino,586.0,43.0,64.0,12750.0,337.254902,501.960784,10.0
Minas de Riotinto,127.0,10.0,10.0,3812.0,262.329486,262.329486,6.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villarrasa,172.0,6.0,16.0,2211.0,271.370421,723.654455,0.0
Niebla,236.0,16.0,25.0,4116.0,388.726919,607.385811,1.0
Almonte,1836.0,76.0,140.0,24507.0,310.115477,571.265353,19.0
Valverde del Camino,586.0,43.0,64.0,12750.0,337.254902,501.960784,10.0
Palma del Condado (La),1152.0,15.0,52.0,10801.0,138.876030,481.436904,3.0
Palos de la Frontera,666.0,32.0,53.0,11742.0,272.525975,451.371146,4.0
Moguer,1292.0,36.0,93.0,21867.0,164.631637,425.298395,3.0
Campofrío,10.0,1.0,3.0,713.0,140.252454,420.757363,0.0
Granada de Río-Tinto (La),3.0,1.0,1.0,243.0,411.522634,411.522634,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Manzanilla,99.0,1.0,7.0,2118.0,47.214353,330.500472,0.0,0.142857
Ayamonte,1317.0,3.0,17.0,21104.0,14.215315,80.553450,0.0,0.176471
Calañas,191.0,2.0,10.0,2768.0,72.254335,361.271676,0.0,0.200000
Zalamea la Real,91.0,1.0,4.0,3054.0,32.743942,130.975769,0.0,0.250000
Lepe,2128.0,6.0,23.0,27880.0,21.520803,82.496413,1.0,0.260870
Palma del Condado (La),1152.0,15.0,52.0,10801.0,138.876030,481.436904,3.0,0.288462
Punta Umbría,812.0,12.0,41.0,15355.0,78.150440,267.014002,4.0,0.292683
Campofrío,10.0,1.0,3.0,713.0,140.252454,420.757363,0.0,0.333333
Cerro de Andévalo (El),146.0,3.0,8.0,2327.0,128.921358,343.790288,0.0,0.375000
